# Battle of the Neighborhoods
## Salon Startups

First, download any necessary libraries; I downloaded all the libraries I normally would, so some are unused.

In [1]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors



print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

Connect to the Foursquare API using your credentials

In [156]:
CLIENT_ID = 'JCM0LKLNUN3ZYQ0CNVNQDSQCDGQ5P2C0PSWN3RPJ1LBOHWY1' # your Foursquare ID
CLIENT_SECRET = 'X1HO1XV4PGI0USKYI11WB1ZY2ZN0BKCICOGZNJZYEZCQSJDS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JCM0LKLNUN3ZYQ0CNVNQDSQCDGQ5P2C0PSWN3RPJ1LBOHWY1
CLIENT_SECRET:X1HO1XV4PGI0USKYI11WB1ZY2ZN0BKCICOGZNJZYEZCQSJDS


Create a list of cities to be explored; ie: New York, Austin, San Francisco, and Dallas.
Using the category ID for salons, a dataframe is created with these venues' information.

In [157]:
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Austin, TX', 'San Francisco, CA', 'Dallas, TX']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d110951735")
    results[city] = requests.get(url).json()

Iteratively create maps for each city; zoom level is set to 11, 12 can also be used but in bigger cities such as Austin, TX, a handful of pizza places cannot be seen.
This way, we simply call maps[city#], and the map for that city is created.

In [88]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of salons in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of salons in New York, NY =  234
Showing Top 100
Total number of salons in Austin, TX =  174
Showing Top 100
Total number of salons in San Francisco, CA =  203
Showing Top 100
Total number of salons in Dallas, TX =  179
Showing Top 100


## Salons in New York City, NY

In [89]:
maps[cities[0]]

## Salons in Austin, TX

In [90]:
maps[cities[1]]

## Salons in San Francisco, CA

In [91]:
maps[cities[2]]

## Salons in Dallas, TX

In [92]:
maps[cities[3]]

# Finding Salon Venue Density per City

We now find the average distance of each salon by using the average coordinate location of the list. 
Each map is again iteratively created, and each average is printed out, showing the average distance to this point. 
The smaller the number, the more dense these veneus are places. The larger the number, the farther apart they are. 

In [109]:
maps = {}
avgDist = []
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=12)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="purple", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    avgDist.append(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))
    print("Average Distance from center point: ", np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)), "\n")


New York, NY
Average Distance from center point:  0.023520295549867166 

Austin, TX
Average Distance from center point:  0.08499424762047512 

San Francisco, CA
Average Distance from center point:  0.02382060652565619 

Dallas, TX
Average Distance from center point:  0.11517732393341974 



## Salons in New York City, NY:  Density

In [110]:
maps[cities[0]]

In [100]:
print("Average distance in New York City: ", avgDist[0])

Average distance in New York City:  0.023520295549867166


## Salons in Austin, TX:  Density

In [101]:
maps[cities[1]]

In [102]:
print("Average distance in Austin: ", avgDist[1])

Average distance in Austin:  0.08499424762047512


## Salons in San Francisco, CA:  Density

In [103]:
maps[cities[2]]

In [104]:
print("Average distance in San Francisco: ", avgDist[2])

Average distance in San Francisco:  0.02382060652565619


## Salons in Dallas, TX:  Density

In [105]:
maps[cities[3]]

In [106]:
print("Average distance in Dallas: ", avgDist[3])

Average distance in Dallas:  0.11517732393341974


Looking at the differences in data, we can see that there are several factors that could influence a business owner who wants to open a salon in deciding which city to do so. New York has the most dense salon locations, and Dallas has the least dense. In the case of Dallas, as a new business owner they would not have as much competition as they may have in New York. But there may be a reason that these venues are less dense in Dallas. It may be because there is less people, or less popularity in salons, and this would have to be further explored and analyzed. 